<h2>Coleta de dados para a base<h2>
_________________________

Problema - Arrumar um modo de definir quando a coleta é ate o final do video e quando tem que colertar por frame

In [ ]:
alimentaDataframeBase("../Videos/Dormindo/",pd.read_csv('Base_Normal.cvs'))

In [2]:
import cv2
import dlib
import pandas as pd

import os

In [ ]:
#dataframe_base = pd.read_csv('Base_Normal.cvs') #carrega dataframe base
#dataframe_base = pd.read_csv('Base_Dormindo.cvs') #carrega dataframe base

In [ ]:
detector = dlib.get_frontal_face_detector() #Face detector
predictor = dlib.shape_predictor("../Dependencias/shape_predictor_68_face_landmarks.dat") #preve os pontos do rosto
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8)) #cria uma configuração para tratar imagem a partir do contrates

In [ ]:
#Função recebe dataframe base e o caminho dos videos que será coletado os dados
def alimentaDataframeBase(caminho_videos,dataframe,limit=-1):
    for arquivo in os.listdir(caminho_videos): #passa pegando os nomes dos videos
        dataframe.append(coletaDados(caminho_videos+arquivo,limit),ignore_index=True) #dataframe base recebe os dados coletados do video

In [ ]:
def coletaDados(arquivo_video,qtde_frame_Limite):

    video_capture = cv2.VideoCapture(arquivo_video) #carrega o video na variavel

    dadosDoVideoLinha = pd.DataFrame() #criar data frame vazio
    dadosDoVideoDataframe = pd.DataFrame() #criar data frame vazio
    currentFrame = 0       #controle de frame
    semRosto = 0           #controle de frame sem rosto indentificado

    while video_capture.grab(): #enquanto tiver frames
        if (video_capture.get(cv2.CAP_PROP_FRAME_COUNT) == currentFrame):
            return dadosDoVideoLinha
            
        currentFrame+=1

        ret, frame = video_capture.retrieve()          #pega proximo quadro
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #passa a imagem para cinza
        clahe_image = clahe.apply(gray)                #aplica uma configuração para melhor detecção
        #cv2.imshow("image", frame) #mostra a imagem

        detections = detector(clahe_image, 1) #detecta as faces na imagem

        if detections:
            for k,d in enumerate(detections): #For each detected face
                shape = predictor(clahe_image, d) #Get coordinates
                for i in range(0,68): #There are 68 landmark points on each face
                    dadosDoVideoLinha = pd.concat([dadosDoVideoLinha,pd.DataFrame({str(i+1)+'.x':[shape.part(i).x] ,str(i+1)+'.y':[shape.part(i).y]})],axis=1)
        else:
            semRosto = semRosto+1
            
        if(qtde_frame_Limite > -1 and qtde_frame_Limite == currentFrame): #se foi definido qtde_frame_Limite e ele foi atingido
            qtde_frame_Limite*2 #dobra limite para o proximo corte
            dadosDoVideoDataframe.append(dadosDoVideo,ignore_index=True) #Coleta dados da primeira linha no dataframe final
            dadosDoVideoLinha = pd.DataFrame() #zera o data frame linha

    video_capture.release() #fecha o arquivo do video carregado
    
    if(qtde_frame_Limite < 0):
        return dadosDoVideoLinha
    else:
        return dadosDoVideoDataframe

In [53]:
i=3
#linha2 = pd.DataFrame({str(i)+'.x':[3] ,str(i)+'.y':[2]})
dataframe_base = pd.DataFrame()
dataframe_Video = pd.DataFrame()

dataframe_Video = pd.concat([dataframe_Video,pd.DataFrame({str(i)+'.a':[5] ,str(i)+'.b':[4]})],axis=1)

#linha2 = pd.DataFrame({str(i)+'.a':[5] ,str(i)+'.b':[4]})
#linha1 = pd.DataFrame({str(i)+'.a':[5] ,str(i)+'.b':[4]})

linha1 = pd.concat([linha1, pd.DataFrame({str(i)+'.a':[5] ,str(i)+'.b':[4]})],axis=1)

dataframe_base = dataframe_base.append(dataframe_Video,ignore_index=True)

In [54]:
dataframe_base

,3.a,3.b
0,5,4


In [ ]:
linha1.to_csv("output.csv")

In [ ]:
linha = pd.DataFrame() #criar data frame vazio
currentFrame = 1
semRosto = 0
while video_capture.grab(): #enquanto tiver quadros

    currentFrame=currentFrame+1 #conta os frames
    
    ret, frame = video_capture.retrieve() #pega prximo quadro
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #passa a imagem para cinza
    clahe_image = clahe.apply(gray) #aplica a configuração para melhor detecção
    #cv2.imshow("image", frame) #mostra a imagem
    
    detections = detector(clahe_image, 1) #detecta as faces na imagem

    if detections:
        for k,d in enumerate(detections): #For each detected face  
            shape = predictor(clahe_image, d) #Get coordinates
            for i in range(0,68): #There are 68 landmark points on each face
            #cv2.circle(frame, (shape.part(i).x, shape.part(i).y), 1, (0,255,0), thickness=-1) #For each point, draw a red circle with thickness2 on the original frame            
                ##linha = linha.append(pd.Series([shape.part(i).x,shape.part(i).y]),ignore_index=True) #adiciona as cordenadas no vetor
                linha = pd.concat([linha,pd.DataFrame({str(i+1)+'.x':[shape.part(i).x] ,str(i+1)+'.y':[shape.part(i).y]})],axis=1) 

    #cv2.putText(frame, str(shape.part(1).x), (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
    else:
        semRosto = semRosto+1

        
dataframe_base.append(linha,ignore_index=True) #adiciona um linha com os valores do vetor

video_capture.release()

In [ ]:
video_capture.get(cv2.CAP_PROP_POS_FRAMES) #frame atual do video

In [ ]:
video_capture.get(cv2.CAP_PROP_FRAME_COUNT) #total de frames do video

In [ ]:

Total de Frame do video 

160

5 seg de 30fps
150 frames no total


total de frame - current frame > 150

toral de frames restante tem que dar mais 5 segundos de video


Se for varios videos
coleta os 5 segundo
adiciona no dataframe base
e vai pro proximo...

Se for video grande
coleta os 5 segundos
valida se tem frame suficiente pra mais um ciclo
se tem
   adiciona no datafrmae base
   limpa data set linha
   continua coleta
Se ñ tem
    adiciona no datafrmae base
e acaba...

Variaveis
segundos de coleta. 
fps considerado.
dataset base
caminho dos ou do video